In [1]:
import numpy as np
import os
import glob
import csv
import re
import pandas as pd
from pprint import pprint
from pathlib import Path

In [2]:
def parse_hw_csv_2(csv_file, hw_data, appargs, kdata):
    cfg = None

    return
    with open(csv_file, 'rU') as data_file:
        print("Parsing HW csv file {0}".format(csv_file))
        reader = csv.reader(data_file)
        state = "start"
        header = []
        kcount = 0
        for row in reader:
            if state == "start":
                if len(row) == 0:
                    continue
                if "ID" == row[0]:
                    state = "kernel_proc"
                continue
            if state == "kernel_proc":
                if len(row) == 1:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    break
                metric = row[-3]
                if metric == "device__attribute_display_name":
                    cfg = row[-1]
                    continue

                try:
                    value = float(row[-1].replace(",",""))
                except ValueError:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    continue

                if len(kdata) <= kcount:
                    kdata.append({})
                if metric not in kdata[kcount]:
                    kdata[kcount][metric] = []
                kdata[kcount][metric].append(value)
                kcount = int(row[0]) + 1

        print("Kernels found: {0}".format(kcount))

    if cfg != "" and cfg != None:
        cfg = re.sub(r" \(\d+\)$","", cfg) # Strip off the (0), (1), etc that some profiler versions put on the end of the device name
        if cfg not in hw_data:
            hw_data[cfg] = {}
        hw_data[cfg][appargs] = kdata

In [25]:
def parse_hw_csv(csv_file, appargs, kdata):
    cfg = ""
    cfg_col = None
    hw_data = {}

    with open(csv_file, 'rU') as data_file:
        print("Parsing HW csv file {0}".format(csv_file))
        
        # print(data_file.read())
        # return
        
        # line must match r"^==\d*==\s*Profiling result:\s*$"
        # line must match r"^==PROF== Disconnected\s*$"
        
        reader = csv.reader(data_file)
        state = "start"
        header = []
        kcount = 0
        for row in reader:
            print(row)
            # Begin by searching for the text line that indicates the beginning of
            # the profile dump
            if state == "start" and len(row) > 0:
                if "Profiling result" in row[0] or "==PROF== Disconnected" in row[0]:
                    state = "header_proc"
                continue

            # The frist line is a header line what indicates the place of each
            # stat on the next line
            if state == "header_proc":
                if "Event result" in row[0]:
                    continue
                header = row
                count = 0

                # get the device name column - which is a special attribute
                for elem in row:
                    if elem == "Device":
                        cfg_col = count
                    elif elem == "device__attribute_display_name":
                        cfg_col = count
                    count += 1

                state = "kernel_proc"
                continue

            # The next sequence of lines are all the kernel launches with the values for each stat
            if state == "kernel_proc":
                if len(row) == 1:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    break

                # skip the memcopies
                if "[CUDA " in "".join(row):
                    continue

                if cfg_col == None:
                    continue

                # Skip lines without a device listed
                if row[cfg_col] == "":
                    continue

                # Set the Device
                if cfg != "" and cfg != row[cfg_col]:
                    print("data for more than one device in {0}..{1}:{2}"\
                        .format(csv_file,cfg,elem))
                    exit()

                cfg = row[cfg_col]
                count = 0
                if len(kdata) <= kcount:
                    kdata.append({})
                for elem in row:
                    if header[count] not in kdata[kcount]:
                        kdata[kcount][header[count]] = []
                    print("header[count]", header[count])
                    print("kdata[kcount][header[count]", kdata[kcount][header[count]])
                    try:
                        value = float(elem.replace(",",""))
                        kdata[kcount][header[count]].append(value)
                    except ValueError:
                        if "n/a" != elem:
                            kdata[kcount][header[count]].append(elem)
                    count += 1
                    
#                logger.log("Kernel Launch {0}: HW Kernel found".format(kcount))
                kcount += 1
                continue
        
        print("Kernels found: {0}".format(kcount))
        
    if cfg != "" and cfg != None:
        # Strip off the (0), (1), etc that some profiler versions 
        # put on the end of the device name
        cfg = re.sub(r" \(\d+\)$","", cfg)
        if cfg not in hw_data:
            hw_data[cfg] = {}
        hw_data[cfg][appargs] = kdata
        
    return hw_data

In [26]:
print("Getting HW data")
hw_data = {}

root_path = Path("/home/roman/dev/gpgpusims")
hw_path = root_path / "analyze" / "hw_run"
options = {
    "hardware_dir": hw_path
}


Getting HW data


In [27]:
def run():
    for root, dirs, files in os.walk(options["hardware_dir"]):
        for d in dirs:
            csv_dir = os.path.join(root, d)
            csv_files = sorted(glob.glob(os.path.join(csv_dir,"*.csv*")))
            if len(csv_files) == 0:
                continue
            # latest_date = re.search("(.*).csv*",os.path.basename(csvs[-1])).group(1)
            # csvs = glob.glob(os.path.join(csv_dir,"{0}.csv*".format(latest_date)))
            # logger.log("For {0}: Using Date: [{1}]. Containd {2} files\n".format(csv_dir, latest_date, len(csvs)))
            kdata = []
            for csv_file in csv_files:
                app_args = os.path.join(os.path.basename(root), d)
                if "gpc__cycles_elapsed" in csv_file:
                    raise ValueError("todo")
                    new_hw_data = parse_hw_csv_2(csv_file, app_args, kdata)
                    df = pd.DataFrame.from_records(new_hw_data)
                    return df
                else:
                    new_hw_data = parse_hw_csv(csv_file, app_args, kdata)
                    data = new_hw_data['NVIDIA GeForce GTX 1080']['pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt']
                    pprint(data)
                    df = pd.DataFrame.from_records(data)
                    return df

run()

Parsing HW csv file /home/roman/dev/gpgpusims/analyze/hw_run/device-0/10.1/pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0
['==219== NVPROF is profiling process 219', ' command: /apps/src/..//bin/10.1/release/pathfinder-rodinia-2.0-ft 1000 20 5 ./data/result_1000_20_5.txt']
['==219== Profiling application: /apps/src/..//bin/10.1/release/pathfinder-rodinia-2.0-ft 1000 20 5 ./data/result_1000_20_5.txt']
['==219== Profiling result:']
['Start', 'Duration', 'Grid X', 'Grid Y', 'Grid Z', 'Block X', 'Block Y', 'Block Z', 'Registers Per Thread', 'Static SMem', 'Dynamic SMem', 'Size', 'Throughput', 'SrcMemType', 'DstMemType', 'Device', 'Context', 'Stream', 'Name', 'Correlation_ID']
['us', 'us', '', '', '', '', '', '', '', 'KB', 'B', 'KB', 'GB/s', '', '', '', '', '', '', '']
['295600.155000', '1.056000', '', '', '', '', '', '', '', '', '', '3.906250', '3.527737', 'Pageable', 'Device', 'NVIDIA GeForce GTX 1080 (0)', '1', '7', '[CUDA memcpy 

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,SrcMemType,DstMemType,Device,Context,Stream,Name,Correlation_ID
0,[295643.579],[5.024],[5.0],[1.0],[1.0],[256.0],[1.0],[1.0],[18.0],[2.0],[0.0],[],[],[],[],[NVIDIA GeForce GTX 1080 (0)],[1.0],[7.0],[_Z14dynproc_kerneliPiS_S_iiii],[112.0]
1,[295651.419],[3.392],[5.0],[1.0],[1.0],[256.0],[1.0],[1.0],[18.0],[2.0],[0.0],[],[],[],[],[NVIDIA GeForce GTX 1080 (0)],[1.0],[7.0],[_Z14dynproc_kerneliPiS_S_iiii],[113.0]
2,[295656.571],[3.264],[5.0],[1.0],[1.0],[256.0],[1.0],[1.0],[18.0],[2.0],[0.0],[],[],[],[],[NVIDIA GeForce GTX 1080 (0)],[1.0],[7.0],[_Z14dynproc_kerneliPiS_S_iiii],[114.0]
3,[295661.883],[2.976],[5.0],[1.0],[1.0],[256.0],[1.0],[1.0],[18.0],[2.0],[0.0],[],[],[],[],[NVIDIA GeForce GTX 1080 (0)],[1.0],[7.0],[_Z14dynproc_kerneliPiS_S_iiii],[115.0]


In [44]:
csv_files_final = []
for root, dirs, files in os.walk(options["hardware_dir"]):
    for d in dirs:
        csv_dir = os.path.join(root, d)
        csv_files = sorted(glob.glob(os.path.join(csv_dir,"*.csv*")))
        if len(csv_files) == 0:
            continue
        for csv_file in csv_files:
            csv_files_final.append(csv_file)
csv_files_final = [str(Path(f).relative_to(root_path)) for f in csv_files_final]
pprint(csv_files_final)
pprint(len(csv_files_final))

['analyze/hw_run/device-0/10.1/pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/backprop-rodinia-2.0-ft/4096___data_result_4096_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/backprop-rodinia-2.0-ft/4096___data_result_4096_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/bfs-rodinia-2.0-ft/__data_graph4096_txt___data_graph4096_result_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/bfs-rodinia-2.0-ft/__data_graph4096_txt___data_graph4096_result_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/srad_v2-rodinia-2.0-ft/__data_matrix128x128_txt_0_127_0_127__5_2___data_result_matrix128x128_1_150_1_100__5_2_txt/22.11.3

In [47]:
csv_files_final = list(options["hardware_dir"].rglob("*.csv*"))
csv_files_final = [str(Path(f).relative_to(root_path)) for f in csv_files_final]
pprint(csv_files_final)
pprint(len(csv_files_final))

['analyze/hw_run/device-0/10.1/pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/pathfinder-rodinia-2.0-ft/1000_20_5___data_result_1000_20_5_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/backprop-rodinia-2.0-ft/4096___data_result_4096_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/backprop-rodinia-2.0-ft/4096___data_result_4096_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/bfs-rodinia-2.0-ft/__data_graph4096_txt___data_graph4096_result_txt/22.11.30-Wednesday--00:59:36.csv.elapsed_cycles_sm.0',
 'analyze/hw_run/device-0/10.1/bfs-rodinia-2.0-ft/__data_graph4096_txt___data_graph4096_result_txt/22.11.30-Wednesday--00:59:36.csv.cycle.0',
 'analyze/hw_run/device-0/10.1/srad_v2-rodinia-2.0-ft/__data_matrix128x128_txt_0_127_0_127__5_2___data_result_matrix128x128_1_150_1_100__5_2_txt/22.11.3

In [12]:
summarize_hw_data(hw_data)

NameError: name 'summarize_hw_data' is not defined